In [20]:
import cv2
import numpy as np
import os
from tqdm import tqdm
from PIL import Image
import json

In [21]:
MARKUP_FILE = "../data/background_markup.json"

with open(MARKUP_FILE, 'r') as input_file:
    backgrounds_markup = json.load(input_file)["data"]

In [23]:
PAPERS_FOLDER_PATH = "../data/images/"
BACKGROUNDS_FOLDER_PATH = "../data/backgrounds/"

images = os.listdir(PAPERS_FOLDER_PATH)
backgrounds = os.listdir(BACKGROUNDS_FOLDER_PATH)

for image_name in tqdm(images):
    for background_name in tqdm(backgrounds):
        if background_name != 'art_gallery.jpg':
            continue
        image = cv2.imread(os.path.join(PAPERS_FOLDER_PATH, image_name))
        background = cv2.imread(os.path.join(BACKGROUNDS_FOLDER_PATH, background_name))

        background = cv2.cvtColor(background, cv2.COLOR_RGB2RGBA)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2RGBA)
        
        background_data = list(filter(lambda background: background["image_name"] == background_name, backgrounds_markup))[0]

        background_rows = background.shape[0]
        background_columns = background.shape[1]

        rows = image.shape[0]
        columns = image.shape[1]

        init = np.float32([[0, 0], [0, rows - 1], [columns - 1, rows - 1], [columns - 1, 0]])
        res = np.float32(background_data["borders"])

        M = cv2.getPerspectiveTransform(res, init)
        M_inv = np.linalg.pinv(M)

        result_image = cv2.warpPerspective(image, M_inv, (background_columns, background_rows), cv2.INTER_LINEAR_EXACT)
        
        result_background = background.copy()

        for i in range(result_image.shape[0]):
            for j in range(result_image.shape[1]):
                for channel in range(3):
                    if result_image[i, j, 3] != 0:
                        result_background[i, j, channel] = result_image[i, j, channel]
                        
        cv2.imwrite(f"{image_name}_{background_name}.png", result_background)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [14:27<00:00, 86.76s/it]


In [7]:
image = cv2.imread("1.png_art_and_museum.jpg.png")

In [18]:
kernel = np.ones((2,2),np.float32)/4
dst = cv2.filter2D(image, -1, kernel)

In [19]:
cv2.imwrite("blur.png", dst)

True